# Electromagnetic Fault Injection using the ChipSHOUTER PicoEMP

In this notebook we provide a basic example of how you can use the open-source [ChipSHOUTER PicoEMP EMFI](https://github.com/newaetech/chipshouter-picoemp) tool to inject faults in a LAUNCHXL-CC26x2R1 development board.

Note that most of the provided notebooks used the LAUNCHXL-CC2640R2 development board instead. The reason is simple, we seemed to get a higher success rate with our specific build of the PicoEMP and [injection tip](https://github.com/newaetech/chipshouter-picoemp/tree/main/hardware/injection_tips) on the LAUNCHXL-CC26x2R1, your mileage may vary.

In this notebook we will automate FI attempts by controlling the PicoEMP using a ChipWhisperer. This allows us to precisely time the offset in time (from a trigger signal) when to inject the EM pulse. Safety was one of the main design considerations for the ChipSHOUTER PicoEMP, but connecting your ChipWhisperer to your own homemade PicoEMP is not without risk. Tread carefully.

Alternatively you can also try to inject faults by pressing the PicoEMP buttons manually, by using the fast_trigger mode or by modifying the Raspberry Pi Pico firmware to allow for a configurable delay.


## Hardware Setup

One of the main advantages of EMFI is that you do not have to modify the target development board!
To run this notebook you will need a ChipWhisperer, your ChipSHOUTER PicoEMP and a target development board.

* Remove the 3V3 jumper and connect the target side pin to the ChipWhisperer's 3V3 output
* Remove the RESET jumper and connect the target side to the ChipWhisperer's NRST output
* Connect the ChipWhisperer's IO4/TRG to the target's DIO6 pin
* Connect the ChipWhisperer's ground to a ground pin on the target board

* Connect the ChipWhisperer's IO3 to the PicoEMP's CHG pin
* Connect the ChipWhisperer's HS2 to the PicoEMP's HVP pin
* Connect the ChipWhisperer's GND pin to the PicoEMP's GND pin

![PicoEMP setup](img/picoemp_resize.jpg)

## Preparation

The following cells load the required libraries and initialise the ChipWhisperer as well as our target.

In [1]:
import sys
import time
import os
import numpy as np
import chipwhisperer as cw
from tqdm.notebook import tqdm
import serial
import matplotlib.pyplot as plt

ser = 0

In [2]:
# Connect to the ChipWhisperer and perform some basic initialization

scope = cw.scope()

scope.clock.clkgen_src = 'system' 
scope.clock.clkgen_freq = 200e6          # Main ChipWhisperer clock
scope.clock.adc_mul = 0
scope.trigger.triggers = 'tio4'          # Trigger on a rising edge of TIO4 (connected to DIO6)
scope.adc.basic_mode = 'rising_edge'
scope.io.target_pwr = True

scope.glitch.enabled = True
scope.glitch.clk_src = 'pll'
#scope.clock.pll.update_fpga_vco(600e6)
scope.glitch.output = 'enable_only'
scope.glitch.trigger_src = 'ext_single'
scope.io.glitch_hp = False
scope.glitch.ext_offset = 300            # Glitch offset from the external trigger (in cycles of the main CW clock)

In [3]:
# For use with the ChipShouter Pico

# Disable the glitch mosfet and route the glitch signal to hs2
# Connect HS2 to the ChipShouter Pico HVP pin
scope.io.glitch_lp = False  
scope.io.hs2 = "glitch"
scope.glitch.repeat = 100 # You might want to try different values for this parameter

# Set tio3 to be an input
# Connect tio3 to the CHG pin, this allows us to check if the ChipShouter is charged
scope.io.tio3 = 'high_z'

print(scope.io.tio_states[2])

1


In [4]:
# Simple loop that blocks until the ChipShouter is charged
def wait_for_hv():
    while scope.io.tio_states[2] != 0:
        time.sleep(0.1)

## ChipShouter Pico

In [5]:
# A very basic class to interact with the ChipShouter PicoEMP

class ChipShouterPicoEMP:
    def __init__(self, port='/dev/ttyACM1'):
        self.pico = serial.Serial(port, 115200)
        
        self.pico.write(b'\r\n')
        time.sleep(0.1)
        ret = self.pico.read(self.pico.in_waiting)
        
        if b'PicoEMP Commands' in ret:
            print('Connected to ChipSHOUTER PicoEMP!')
        else:
            raise OSError('Could not connect to ChipShouter PicoEMP :(')
        

    def disable_timeout(self):
        self.pico.write(b'disable_timeout\r\n')
        time.sleep(1)
        assert b'Timeout disabled!' in self.pico.read(self.pico.in_waiting)

        
    def arm(self):
        self.pico.write(b'arm\r\n')
        time.sleep(1)
        assert b'Device armed' in self.pico.read(self.pico.in_waiting)

        
    def disarm(self):
        self.pico.write(b'disarm\r\n')
        time.sleep(1)
        assert b'Device disarmed!' in self.pico.read(self.pico.in_waiting)

        
    def external_hvp(self):
        self.pico.write(b'external_hvp\r\n')
        time.sleep(1)
        assert b'External HVP mode active' in self.pico.read(self.pico.in_waiting)

        
    def print_status(self):
        self.pico.write(b'status\r\n')
        time.sleep(1)
        print(self.pico.read(self.pico.in_waiting).decode('utf-8'))
        
    
    def setup_external_control(self):
        self.disable_timeout()
        self.external_hvp()
        self.print_status()

In [6]:
# Change the serial port if needed!
pico = ChipShouterPicoEMP('/dev/ttyACM5')
pico.setup_external_control()

Connected to ChipSHOUTER PicoEMP!
status
Status:
- Disarmed
- Not charged
- Timeout disabled
- HVP external

> 



In [7]:
# Connect to the LAUNCHXL-CC26X2R1 UART
# You may have to change the serial port ('/dev/ttyACM1')

if ser:
    ser.close()

ser = serial.Serial('/dev/ttyACM1', 115200)

In [8]:
# Modify the dslite_path variable to point to your installation of Uniflash
# Running this cell will load the example target firmware
# THIS WILL OVERWRITE THE FIRMWARE ON YOUR LAUNCHXL-CC26X2R1

import subprocess
from pathlib import Path

home_dir = str(Path.home()) 
dslite_path = home_dir + '/ti/uniflash_7.0.0/dslite.sh'
erase_cmd = dslite_path + ' --mode cc13xx-cc26xx-mass-erase -d XDS110'

# Note that the following command is different compared to the previous notebooks
flash_cmd = dslite_path + ' --config ./bin/CC2652R1F.ccxml --flash ./bin/VFI_SCA_CC2652R1.out' 

process = subprocess.Popen(erase_cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output = process.communicate()

if b'Device Unlocked' not in output[0]:
    print('There was an error while trying to erase the microcontroller')
    print(output)
else:
    scope.io.nrst = 'low'
    scope.io.target_pwr = False
    time.sleep(0.1)
    scope.io.target_pwr = True
    scope.io.nrst = 'high'
    
    process = subprocess.Popen(flash_cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = process.communicate()
    if b'Board Reset Complete' in output[0]:
        print('Target has been flashed!')
    else:
        print('Error flashing target. Check your connections and try again.')

Target has been flashed!


In [9]:
# Simple function to reset the target microcontroller
def reset_dut(delay=0.1):
    scope.io.nrst = 'low'
    scope.io.target_pwr = False
    time.sleep(delay)
    scope.io.target_pwr = True
    scope.io.nrst = 'high'
    time.sleep(0.05)
    ser.flushInput()
    ser.write(b'd') # To select the double loop function of the firmware
    

# A more thorough reset function that verifies that the target is alive again
def thorough_reset_dut(delay=0.05): 
    reset_dut(delay)
    
    ser.flushInput()
    ser.write(bytes([0xAA]))
    time.sleep(0.1)
    ret = ser.read(ser.in_waiting)

    while int.from_bytes(ret, 'little') != 10000:
        delay += 0.5
        reset_dut(delay)
        ser.write(bytes([0xAA]))
        time.sleep(0.1)
        ret = ser.read(ser.in_waiting)

In [10]:
reset_dut()

In [11]:
ser.write(bytes([0xAA]))
time.sleep(0.1)
ret = ser.read(ser.in_waiting)
print(ret, int.from_bytes(ret, 'little'))

b"\x10'\x00\x00" 10000


## A basic experiment

The following cell will control the ChipWhisperer and PicoEMP to inject EM pulses into the target.

We assume the target crashed if it does not return any output. We successfully injected a glitch if the target replies with a counter value that is not equal to 10000, those values will be printed. In both cases we reset the target device.

The main parameter you will have to tune is the PicoEMP positioning relative to the target. It can be tricky to find a good location, but once you hit the spot the success rate can be pretty high!

Assuming you got this basic experiment to work and completed all of the other notebooks, you should be able to extend this notebook to try and bypass code readout protection using EMFI!

In [12]:
scope.glitch.repeat = 35
offsets = np.arange(0, 500, 20)
repeats = 10
crashes = 0
faults = 0

pico.arm()

for offset in tqdm(range(len(offsets))):
    scope.glitch.ext_offset = offsets[offset]
    
    for i in range(repeats):
        wait_for_hv() # Wait for the ChipShouter to be charged
        
        scope.arm()
        ser.write(bytes([0xAA]))
        time.sleep(0.1)
        ret = ser.read(ser.in_waiting)
        
        if ret != b"\x10'\x00\x00":
            if ret == b'':
                crashes += 1
                print('.', end = '')
            else:
                faults += 1
                print('Fault?!', int.from_bytes(ret, 'little'), ret.hex(), offsets[offset])
            
            thorough_reset_dut() 
            
    if offset % 20 == 0:
        print('\n', crashes, faults)
                
pico.disarm()

total = len(offsets)*repeats
print("\nTotal # attempts:", total) 
print("Total # faults: %d (%f%%)" % (faults, (faults/total)*100))
print("Total # crashes: %d (%f%%)" % (crashes, (crashes/total)*100))

  0%|          | 0/25 [00:00<?, ?it/s]

Fault?! 2506106555 bb266095 0

 0 1
....Fault?! 9999 0f270000 60
Fault?! 9999 0f270000 60
Fault?! 9999 0f270000 60
Fault?! 9999 0f270000 80
.Fault?! 9999 0f270000 80
.Fault?! 536962703 8f660120 80
..Fault?! 9998 0e270000 80
.Fault?! 9999 0f270000 80
.Fault?! 9999 0f270000 140
.Fault?! 9999 0f270000 140
Fault?! 9999 0f270000 140
............Fault?! 9998 0e270000 280
Fault?! 9999 0f270000 280
Fault?! 536962700 8c660120 280
Fault?! 536962700 8c660120 280
Fault?! 9999 0f270000 280
Fault?! 536962700 8c660120 280
Fault?! 9995 0b270000 280
Fault?! 9999 0f270000 280
..Fault?! 9999 0f270000 340
.Fault?! 9999 0f270000 340
Fault?! 9999 0f270000 340
......
 32 23
....Fault?! 9999 0f270000 460
Fault?! 9999 0f270000 460
Fault?! 9999 0f270000 460
.Fault?! 9999 0f270000 480
Fault?! 9999 0f270000 480
Fault?! 9999 0f270000 480
.....
Total # attempts: 250
Total # faults: 29 (11.600000%)
Total # crashes: 42 (16.800000%)


In [ ]:
ser.close()
scope.dis()
pico.disarm()